In [1]:
import pandas as pd
from pyproj import Geod
import itertools
import json

In [2]:
# Figures out how to draw a curved line between two points
# Makes sure that routes to East Asia/Oceania goes over the pacific
# This code is a modified version of https://github.com/kburchfiel/route_maps_builder
long_cutoff = 80
def get_gcpoints(g, row):
    orig_lat, orig_lon = row['lat_A'], row['lon_A']
    if orig_lon > long_cutoff:
        orig_lon -= 360
    dest_lat, dest_lon = row['lat_B'], row['lon_B']
    if dest_lon > long_cutoff:
        dest_lon -= 360
    gc_points = g.npts(orig_lon, orig_lat, dest_lon, dest_lat, 20)
    revised_gc_points = [(orig_lat, orig_lon)]
    for _lon, _lat in gc_points:
        if _lon > long_cutoff:
            _lon -= 360
        revised_gc_points.append((_lat, _lon))
    revised_gc_points.append((dest_lat, dest_lon))
    return revised_gc_points

    
def generate_map(df, map_name):
    
    g = Geod(ellps="WGS84")
    
    # Gets a unique list of airports in the data    
    airports = pd.concat([df[['A', 'lat_A', 'lon_A']].set_axis(['code', 'lat', 'lon'], axis=1),
                          df[['B', 'lat_B', 'lon_B']].set_axis(['code', 'lat', 'lon'], axis=1)
                         ]).drop_duplicates('code').reset_index(drop=True)        
        
    # Creates a json file for all the airports in the dataset
    circlejson = {'type': 'FeatureCollection', 'features':[]}
    for i in range(len(airports)):
        row = airports.loc[i]
        lat, lon = row['lat'], row['lon']
        if row['lon'] > long_cutoff:
            lon -= 360
        circlejson['features'].append({
            'geometry': {'type': 'Point', 'coordinates': [lon,lat]},
            'type': 'Feature',
            'id': row['code']
        })
    with open(f'../data/{map_name}/airports.json', 'w') as f:
        print(json.dumps(circlejson, indent=2), file=f)
        
    # Creates a json file for all the routes in the dataset
    linejson = {'type': 'FeatureCollection', 'features':[]}
    for i in range(len(df)):
        print(f'Line {i+1}/{len(df)}', end='\r')
        row = df.loc[i]
        if row['color'] == 'white': continue
        locs = [[lon,lat] for (lat,lon) in get_gcpoints(g, row)]
        linejson['features'].append({
            'geometry': {'type': 'LineString', 'coordinates': locs},
            'type': 'Feature',
            'id': row['AB'],
            'properties': {'style': {'color': row['color']}}
        })
    with open(f'../data/{map_name}/routes.json', 'w') as f:
        print(json.dumps(linejson, indent=2), file=f)

In [3]:
%%time
all_routes = pd.read_csv('../data/all/routes.csv')
generate_map(all_routes, 'all')

CPU times: user 951 ms, sys: 43.9 ms, total: 995 ms
Wall time: 1.13 s


In [4]:
%%time
aa_routes = pd.read_csv('../data/american/routes.csv')
generate_map(aa_routes, 'american')

CPU times: user 353 ms, sys: 15.1 ms, total: 368 ms
Wall time: 606 ms


In [5]:
%%time
dl_routes = pd.read_csv('../data/delta/routes.csv')
generate_map(dl_routes, 'delta')

CPU times: user 328 ms, sys: 14.1 ms, total: 343 ms
Wall time: 455 ms


In [6]:
%%time
ua_routes = pd.read_csv('../data/united/routes.csv')
generate_map(ua_routes, 'united')

CPU times: user 319 ms, sys: 16.6 ms, total: 336 ms
Wall time: 429 ms
